# Setttings

In [1]:
import numpy as np
import tensorflow as tf

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Read in data

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("../../Tensorflow-applications/MNIST_data//", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../Tensorflow-applications/MNIST_data//train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ../../Tensorflow-applications/MNIST_data//train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting ../../Tensorflow-applications/MNIST_data//t10k-images-idx3-ubyte.gz
Extracting ../../Tensorflow-applications/MNIST_data//t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


In [3]:
print(mnist.train.images.shape)
print(mnist.validation.images.shape)
print(mnist.test.images.shape)

(55000, 784)
(5000, 784)
(10000, 784)


In [4]:
mnist.train.labels[:,:].shape

(55000, 10)

# Create graph

### Settings

In [35]:
epsilon = 1e-7
learning_rate = 0.001
batch_size = 50
epochs = 10
rounds = 3

### Input and output tensors

In [36]:
X = tf.placeholder(tf.float32, [None, 784])
X_reshape = tf.reshape(X, [-1, 28, 28, 1])
y = tf.placeholder(tf.float32, [None,10])

## Squash function

In [37]:
def squash(input_vector, axis):
    normalised_input = tf.reduce_sum(tf.square(input_vector), axis = axis, keepdims = True)
    scale = tf.divide(normalised_input, tf.add(normalised_input, 1.))
    vector = tf.divide(input_vector, tf.sqrt(tf.add(normalised_input, epsilon)))
    output = tf.multiply(scale, vector)
    
    return(output)

### Layers

#### Convolutional

In [38]:
conv_filter_shape = [9, 9, 1, 256]
weights = tf.Variable(tf.truncated_normal(conv_filter_shape, stddev=0.3), name='W1')
bias = tf.Variable(tf.truncated_normal([256], stddev=0.3), name='B1')
out_layer = tf.nn.conv2d(X_reshape, weights, [1,1,1,1], padding = 'VALID')
out_layer_bias = tf.add(out_layer, bias)
layer1 = tf.nn.relu(out_layer_bias)

#### PrimaryCaps

In [39]:
conv_filter_shape = [9, 9, 256, 256]
weights = tf.Variable(tf.truncated_normal(conv_filter_shape, stddev=0.3), name='W2')
bias = tf.Variable(tf.truncated_normal([256], stddev=0.3), name='B2')
out_layer = tf.nn.conv2d(layer1, weights, [1,2,2,1], padding = 'VALID')
layer2 = tf.add(out_layer, bias)

In [40]:
caps1_raw = tf.reshape(layer2, [-1, 6*6*32, 8], name='caps1_raw')
caps1_output = squash(caps1_raw, axis=-1)

In [41]:
caps1_output_expand = tf.expand_dims(caps1_output, axis=-1)
caps1_output_expand2 = tf.expand_dims(caps1_output_expand, axis=2)
caps1_output_expand2_tiled = tf.tile(caps1_output_expand2, [1,1,10,1,1], name = 'caps1_out_tiled')

In [42]:
weight_matrix = tf.Variable(tf.truncated_normal([1, 1152, 10, 16, 8], stddev=0.1), name='W_matrix')
batch = tf.shape(X)[0]
weight_matrix_tiled = tf.tile(weight_matrix, [batch, 1, 1, 1, 1], name = 'W_matrix_tiled')

In [43]:
caps2_predicted = tf.matmul(weight_matrix_tiled, caps1_output_expand2_tiled, name='caps2_predicted')

### Routing by agreement

In [44]:
raw_weights = tf.zeros([batch, 1152, 10, 1, 1], name = 'raw_weights')

In [45]:
for i in range(rounds):
    routing_weights = tf.nn.softmax(raw_weights, axis=2, name = 'routing_weights' + str(i))
    weighted_predictions = tf.multiply(routing_weights, caps2_predicted, name = 'weighted_predictions' + str(i))
    weighted_sum = tf.reduce_sum(weighted_predictions, axis=1, name = 'weighted_sum' + str(i), keepdims = True)
    caps2_output = squash(weighted_sum, axis=-2)
    
    caps2_output_tiled = tf.tile(caps2_output, [1, 1152, 1, 1, 1], name = 'caps2_output_tiled'+ str(i))
    agreement = tf.matmul(caps2_predicted, caps2_output_tiled, transpose_a = True, name = 'agreement'+ str(i))
    raw_weights = tf.add(raw_weights, agreement, name = 'raw_weights' + str(i))

In [46]:
squared_norm_caps2 = tf.reduce_sum(tf.square(caps2_output), axis=-2, keepdims = False, name='caps2_norm')
caps2_activation = tf.sqrt(tf.add(squared_norm_caps2, epsilon), name = 'caps2_activation')

In [47]:
#y_proba_max = tf.argmax(caps2_activation, axis=2, name = 'y_proba_max')

In [48]:
y_pred = tf.squeeze(caps2_activation, axis=[1,3])

### Los function

In [49]:
m_minus = 0.1
m_plus = 0.9
lambda_ = 0.5

In [50]:
squared_norm_caps2_loss = tf.reduce_sum(tf.square(caps2_output), axis=-2, keepdims = True, name='caps2_norm_loss')
caps2_activation_loss = tf.sqrt(tf.add(squared_norm_caps2, epsilon), name = 'caps2_activation_loss')

In [51]:
present_error_raw = tf.square(tf.maximum(0., tf.subtract(m_plus, caps2_activation_loss)), 
                              name = 'present_error_raw')
present_error = tf.reshape(present_error_raw, shape = [-1, 10], name = 'present_error')

In [52]:
absent_error_raw = tf.square(tf.maximum(0., tf.subtract(caps2_activation_loss, m_minus)), 
                             name = 'absent_error_raw')
absent_error = tf.reshape(absent_error_raw, shape=[-1,10], name = 'absent_error')

In [53]:
L = tf.add(tf.multiply(y, present_error), tf.multiply(tf.multiply(lambda_, tf.subtract(1., y)), absent_error),
          name = 'L')

In [54]:
margin_loss = tf.reduce_mean(tf.reduce_sum(L, axis=1), name = 'margin_loss')

In [55]:
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(margin_loss)

### Make predictions

In [56]:
correct_prediction = tf.equal(tf.argmax(y,axis=1), tf.argmax(y_pred, axis=1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype = tf.float32))

## Init and saver

In [57]:
init_op = tf.global_variables_initializer()
saver = tf.train.Saver()

## Setup training

In [ ]:
restore_checkpoint = True
best_loss_val = np.infty
checkpoint_path = "../../my_capsule_network"

#If run on AWS
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allocator_type = 'BFC'
config.gpu_options.allow_growth=True

config=config

with tf.Session() as sess:
    if restore_checkpoint and tf.train.checkpoint_exists(checkpoint_path):
        saver.restore(sess, checkpoint_path)
    else:
        init_op.run()

    total_batch = int(len(mnist.train.images)/batch_size)
    total_batch_validation = int(len(mnist.validation.images)/batch_size)

    for epoch in range(epochs):
        avg_cost = 0
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size=batch_size)
            a, c= sess.run([optimizer, margin_loss], feed_dict={X: batch_x, y: batch_y})
            print('batch_iteration:', i)
            avg_cost += c/total_batch
        print("Epoch:", (epoch + 1), "cost =", "{:.3f}".format(avg_cost))
        
        acc_vals = []
        for iterations in range(total_batch_validation):
            batch_x, batch_y = mnist.validation.next_batch(batch_size=batch_size)
            val_acc = sess.run(accuracy, feed_dict={X: batch_x, y: batch_y})
            acc_vals.append(val_acc)
        acc_val = np.mean(acc_vals)
        print('val_acc:', acc_val)
        
        # And save the model if it improved:
        if acc_val < best_loss_val:
            save_path = saver.save(sess, checkpoint_path)
            best_loss_val = acc_val

INFO:tensorflow:Restoring parameters from ../../my_capsule_network
batch_iteration: 0
batch_iteration: 1
batch_iteration: 2
batch_iteration: 3
batch_iteration: 4
batch_iteration: 5
batch_iteration: 6
batch_iteration: 7
batch_iteration: 8
batch_iteration: 9
batch_iteration: 10
batch_iteration: 11
batch_iteration: 12
batch_iteration: 13
batch_iteration: 14
batch_iteration: 15
batch_iteration: 16
batch_iteration: 17
batch_iteration: 18
batch_iteration: 19
batch_iteration: 20
batch_iteration: 21
batch_iteration: 22
batch_iteration: 23
batch_iteration: 24
batch_iteration: 25
batch_iteration: 26
batch_iteration: 27
batch_iteration: 28
batch_iteration: 29
batch_iteration: 30
batch_iteration: 31
batch_iteration: 32
batch_iteration: 33
batch_iteration: 34
batch_iteration: 35
batch_iteration: 36
batch_iteration: 37
batch_iteration: 38
batch_iteration: 39
batch_iteration: 40
batch_iteration: 41
batch_iteration: 42
batch_iteration: 43
batch_iteration: 44
batch_iteration: 45
batch_iteration: 46
bat